# Advanced Trading Strategies Analysis - 高度な取引戦略分析

このノートブックでは、LME銅先物データを使用して、より高度な取引戦略と市場分析を実施します。

## Analysis Overview - 分析の概要
1. **Spread Trading Strategy** - スプレッド取引戦略分析
2. **Seasonality Analysis** - 季節性・循環性分析
3. **Correlation Structure** - 相関構造分析
4. **Risk Parity Analysis** - リスク・パリティ分析
5. **Anomaly Detection** - 異常検知・レジーム分析

In [ ]:
from config.database_config import get_connection_string
import sys
import os
import pandas as pd
import numpy as np
import pyodbc
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.gridspec import GridSpec
import seaborn as sns
from scipy import stats
import warnings

# プロジェクトルートをPythonパスに追加
project_root = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))
sys.path.insert(0, project_root)


warnings.filterwarnings('ignore')

# 英語フォント設定
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['font.size'] = 10
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['figure.dpi'] = 120

# Color palette
COLORS = {
    'primary': '#1f77b4',
    'secondary': '#ff7f0e',
    'tertiary': '#2ca02c',
    'quaternary': '#d62728',
    'quinary': '#9467bd'
}

print("Libraries loaded successfully")

## Data Loading - データ読み込みと前処理

In [ ]:
def load_futures_data(conn, days=365):
    """Load copper futures data"""
    query = f"""
    SELECT 
        p.TradeDate,
        t.TenorTypeName,
        p.SettlementPrice as ClosePrice,
        p.Volume,
        p.OpenInterest,
        CASE 
            WHEN t.TenorTypeName LIKE 'Generic 1st%' THEN 1
            WHEN t.TenorTypeName LIKE 'Generic 2nd%' THEN 2
            WHEN t.TenorTypeName LIKE 'Generic 3rd%' THEN 3
            WHEN t.TenorTypeName LIKE 'Generic 4th%' THEN 4
            WHEN t.TenorTypeName LIKE 'Generic 5th%' THEN 5
            WHEN t.TenorTypeName LIKE 'Generic 6th%' THEN 6
            WHEN t.TenorTypeName LIKE 'Generic 7th%' THEN 7
            WHEN t.TenorTypeName LIKE 'Generic 8th%' THEN 8
            WHEN t.TenorTypeName LIKE 'Generic 9th%' THEN 9
            WHEN t.TenorTypeName LIKE 'Generic 10th%' THEN 10
            WHEN t.TenorTypeName LIKE 'Generic 11th%' THEN 11
            WHEN t.TenorTypeName LIKE 'Generic 12th%' THEN 12
            ELSE NULL
        END as TenorNumber
    FROM T_CommodityPrice p
    INNER JOIN M_Metal m ON p.MetalID = m.MetalID
    INNER JOIN M_TenorType t ON p.TenorTypeID = t.TenorTypeID
    WHERE 
        m.MetalCode = 'COPPER'
        AND p.TradeDate >= DATEADD(day, -{days}, GETDATE())
        AND p.SettlementPrice IS NOT NULL
    ORDER BY p.TradeDate DESC
    """

    df = pd.read_sql(query, conn)
    df['TradeDate'] = pd.to_datetime(df['TradeDate'])
    df = df.dropna(subset=['TenorNumber'])
    return df


# Database connection and data loading
conn = pyodbc.connect(get_connection_string())
df = load_futures_data(conn, days=365)

# Create pivot tables
price_pivot = df.pivot_table(values='ClosePrice', index='TradeDate',
                             columns='TenorNumber', aggfunc='mean').sort_index()
volume_pivot = df.pivot_table(
    values='Volume', index='TradeDate', columns='TenorNumber', aggfunc='mean').sort_index()

# Get available tenors
available_tenors = sorted(
    [col for col in price_pivot.columns if not pd.isna(col)])

print(
    f"Data period: {df['TradeDate'].min().date()} - {df['TradeDate'].max().date()}")
print(f"Data records: {len(df):,}")
print(f"Available tenors: {available_tenors}")
print(f"Price data shape: {price_pivot.shape}")

## 1. Spread Trading Strategy Analysis

### Analysis Objective - 分析目的
- Identify arbitrage opportunities using price differences between different contract months
- Evaluate risk and return of calendar spread strategies

### Methodology - 分析手法
- Calendar spread calculation (near vs far months)
- Statistical properties analysis
- Mean reversion verification
- Risk-adjusted return calculation

### Interpretation - グラフの見方・解釈
- Positive spread: Contango (far month > near month)
- Negative spread: Backwardation (near month > far month)
- Spread volatility: Trading opportunity variation
- Mean reversion coefficient: Spread stability

In [ ]:
# Spread analysis
if len(available_tenors) >= 2:
    near_month = available_tenors[0]
    far_month = available_tenors[1] if len(
        available_tenors) > 1 else available_tenors[0]

    # Calculate spread (far month - near month)
    spread = price_pivot[far_month] - price_pivot[near_month]
    spread = spread.dropna()

    # Spread statistics
    spread_stats = {
        'Mean': spread.mean(),
        'Std Dev': spread.std(),
        'Max': spread.max(),
        'Min': spread.min(),
        'Sharpe Ratio': spread.mean() / spread.std() if spread.std() != 0 else 0
    }

    print(f"\nSpread Statistics (M{far_month} - M{near_month}):")
    for key, value in spread_stats.items():
        print(f"{key}: {value:.2f}")

    # Visualization
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))

    # 1. Spread time series
    axes[0, 0].plot(spread.index, spread.values, linewidth=1, color='blue')
    axes[0, 0].axhline(y=0, color='red', linestyle='--', alpha=0.7)
    axes[0, 0].set_title(f'Calendar Spread (M{far_month} - M{near_month})')
    axes[0, 0].set_ylabel('Spread (USD)')
    axes[0, 0].grid(True, alpha=0.3)

    # 2. Spread distribution
    axes[0, 1].hist(spread.values, bins=30, alpha=0.7,
                    color='green', edgecolor='black')
    axes[0, 1].axvline(x=0, color='red', linestyle='--', alpha=0.7)
    axes[0, 1].set_title('Spread Distribution')
    axes[0, 1].set_xlabel('Spread (USD)')
    axes[0, 1].set_ylabel('Frequency')

    # 3. Price time series comparison
    axes[1, 0].plot(price_pivot.index, price_pivot[near_month],
                    label=f'M{near_month}', linewidth=1)
    axes[1, 0].plot(price_pivot.index, price_pivot[far_month],
                    label=f'M{far_month}', linewidth=1)
    axes[1, 0].set_title('Price Evolution by Tenor')
    axes[1, 0].set_ylabel('Price (USD)')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)

    # 4. Correlation scatter plot
    common_dates = price_pivot.dropna(subset=[near_month, far_month])
    if len(common_dates) > 0:
        axes[1, 1].scatter(common_dates[near_month],
                           common_dates[far_month], alpha=0.6, s=20)
        axes[1, 1].set_xlabel(f'M{near_month} Price (USD)')
        axes[1, 1].set_ylabel(f'M{far_month} Price (USD)')
        axes[1, 1].set_title('Inter-Tenor Correlation')

        # Calculate correlation
        correlation = common_dates[near_month].corr(common_dates[far_month])
        axes[1, 1].text(0.05, 0.95, f'Correlation: {correlation:.3f}',
                        transform=axes[1, 1].transAxes,
                        bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

    plt.tight_layout()
    plt.show()

else:
    print("Spread analysis requires at least 2 tenors")

## 2. Seasonality Analysis

### Analysis Objective - 分析目的
- Identify seasonal patterns and cyclical behavior in copper prices
- Capture predictable price movements

### Methodology - 分析手法
- Monthly and quarterly return analysis
- Day-of-week effect verification
- Moving average deviation analysis

### Interpretation - グラフの見方・解釈
- Monthly returns: Seasonality exists if specific months show bias
- Day-of-week effect: Structural market factors if abnormal returns on specific days
- Cyclical patterns: Regular price movement patterns

In [ ]:
# Seasonality analysis
if len(available_tenors) > 0:
    # Use primary tenor
    main_tenor = available_tenors[0]
    price_series = price_pivot[main_tenor].dropna()

    # Calculate daily returns
    returns = price_series.pct_change().dropna()

    # Add month, weekday, quarter information
    returns_df = pd.DataFrame({
        'returns': returns,
        'month': returns.index.month,
        'weekday': returns.index.weekday,
        'quarter': returns.index.quarter,
        'year': returns.index.year
    })

    # Visualization
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))

    # 1. Monthly average returns
    monthly_returns = returns_df.groupby('month')['returns'].mean()
    month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                   'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

    bars1 = axes[0, 0].bar(range(1, 13), monthly_returns.values,
                           color=[
                               'red' if x < 0 else 'green' for x in monthly_returns.values],
                           alpha=0.7, edgecolor='black')
    axes[0, 0].set_title('Monthly Average Returns')
    axes[0, 0].set_xlabel('Month')
    axes[0, 0].set_ylabel('Average Return')
    axes[0, 0].set_xticks(range(1, 13))
    axes[0, 0].set_xticklabels(month_names, rotation=45)
    axes[0, 0].axhline(y=0, color='black', linestyle='-', alpha=0.3)
    axes[0, 0].grid(True, alpha=0.3)

    # 2. Weekday average returns
    weekday_returns = returns_df.groupby('weekday')['returns'].mean()
    weekday_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

    bars2 = axes[0, 1].bar(range(7), weekday_returns.values,
                           color=[
                               'red' if x < 0 else 'green' for x in weekday_returns.values],
                           alpha=0.7, edgecolor='black')
    axes[0, 1].set_title('Weekday Average Returns')
    axes[0, 1].set_xlabel('Day of Week')
    axes[0, 1].set_ylabel('Average Return')
    axes[0, 1].set_xticks(range(7))
    axes[0, 1].set_xticklabels(weekday_names, rotation=45)
    axes[0, 1].axhline(y=0, color='black', linestyle='-', alpha=0.3)
    axes[0, 1].grid(True, alpha=0.3)

    # 3. Quarterly return distribution
    quarter_data = [returns_df[returns_df['quarter'] == q]
                    ['returns'].values for q in range(1, 5)]
    bp = axes[1, 0].boxplot(quarter_data, labels=[
                            'Q1', 'Q2', 'Q3', 'Q4'], patch_artist=True)

    # Color the boxplots
    colors = ['lightblue', 'lightgreen', 'lightyellow', 'lightcoral']
    for patch, color in zip(bp['boxes'], colors):
        patch.set_facecolor(color)
        patch.set_alpha(0.7)

    axes[1, 0].set_title('Quarterly Return Distribution')
    axes[1, 0].set_xlabel('Quarter')
    axes[1, 0].set_ylabel('Return')
    axes[1, 0].grid(True, alpha=0.3)

    # 4. Cumulative returns seasonality
    yearly_cumulative = returns_df.groupby(['year', 'month'])[
        'returns'].sum().reset_index()
    yearly_pivot = yearly_cumulative.pivot(
        index='month', columns='year', values='returns')

    for year in yearly_pivot.columns:
        if not yearly_pivot[year].isna().all():
            axes[1, 1].plot(yearly_pivot.index, yearly_pivot[year].cumsum(),
                            label=str(year), alpha=0.7, linewidth=1)

    axes[1, 1].set_title('Annual Monthly Cumulative Returns')
    axes[1, 1].set_xlabel('Month')
    axes[1, 1].set_ylabel('Cumulative Return')
    axes[1, 1].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    axes[1, 1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    # Statistical test results
    print("\nSeasonality Analysis Results:")
    print(
        f"Best monthly return: {month_names[monthly_returns.idxmax()-1]} ({monthly_returns.max():.4f})")
    print(
        f"Worst monthly return: {month_names[monthly_returns.idxmin()-1]} ({monthly_returns.min():.4f})")
    print(
        f"Best weekday return: {weekday_names[weekday_returns.idxmax()]} ({weekday_returns.max():.4f})")
    print(
        f"Worst weekday return: {weekday_names[weekday_returns.idxmin()]} ({weekday_returns.min():.4f})")

else:
    print("No data available for seasonality analysis")

## 3. Correlation Structure Analysis

### Analysis Objective - 分析目的
- Analyze correlation relationships between different contract months
- Evaluate portfolio risk management and diversification effects

### Methodology - 分析手法
- Inter-tenor correlation matrix calculation
- Dynamic correlation analysis (rolling correlation)
- Time series changes in correlations

### Interpretation - グラフの見方・解釈
- High correlation (>0.8): Strong price linkage between tenors
- Low correlation (<0.5): Diversification benefits expected
- Correlation changes: Watch for correlation increases during market stress

In [ ]:
# Correlation analysis
if len(available_tenors) >= 2:
    # Calculate daily returns for each tenor
    returns_matrix = price_pivot.pct_change().dropna()

    # Calculate correlation matrix
    correlation_matrix = returns_matrix.corr()

    # Visualization
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))

    # 1. Correlation matrix heatmap
    im1 = axes[0, 0].imshow(correlation_matrix, cmap='RdBu_r', vmin=-1, vmax=1)
    axes[0, 0].set_title('Inter-Tenor Correlation Matrix')
    axes[0, 0].set_xticks(range(len(correlation_matrix.columns)))
    axes[0, 0].set_yticks(range(len(correlation_matrix.columns)))
    axes[0, 0].set_xticklabels(
        [f'M{int(col)}' for col in correlation_matrix.columns], rotation=45)
    axes[0, 0].set_yticklabels(
        [f'M{int(col)}' for col in correlation_matrix.columns])

    # Add correlation coefficients as text
    for i in range(len(correlation_matrix.columns)):
        for j in range(len(correlation_matrix.columns)):
            text = axes[0, 0].text(j, i, f'{correlation_matrix.iloc[i, j]:.2f}',
                                   ha='center', va='center',
                                   color='white' if abs(correlation_matrix.iloc[i, j]) > 0.5 else 'black')

    plt.colorbar(im1, ax=axes[0, 0], shrink=0.8)

    # 2. Rolling correlation (first two tenors)
    if len(available_tenors) >= 2:
        tenor1, tenor2 = available_tenors[0], available_tenors[1]
        rolling_corr = returns_matrix[tenor1].rolling(
            window=30).corr(returns_matrix[tenor2]).dropna()

        axes[0, 1].plot(rolling_corr.index, rolling_corr.values,
                        linewidth=1, color='blue')
        axes[0, 1].set_title(f'Rolling Correlation (M{tenor1} vs M{tenor2})')
        axes[0, 1].set_ylabel('30-day Rolling Correlation')
        axes[0, 1].axhline(y=0, color='red', linestyle='--', alpha=0.5)
        axes[0, 1].axhline(y=0.5, color='orange', linestyle='--', alpha=0.5)
        axes[0, 1].axhline(y=0.8, color='green', linestyle='--', alpha=0.5)
        axes[0, 1].grid(True, alpha=0.3)

    # 3. Principal Component Analysis (if sklearn available)
    try:
        from sklearn.decomposition import PCA
        from sklearn.preprocessing import StandardScaler

        # Data standardization
        scaler = StandardScaler()
        returns_scaled = scaler.fit_transform(returns_matrix)

        # PCA execution
        pca = PCA()
        pca.fit(returns_scaled)

        # Visualization of explained variance
        explained_variance = pca.explained_variance_ratio_
        cumulative_variance = np.cumsum(explained_variance)

        axes[1, 0].bar(range(1, len(explained_variance) + 1), explained_variance,
                       alpha=0.7, color='skyblue', edgecolor='black')
        axes[1, 0].plot(range(1, len(cumulative_variance) + 1), cumulative_variance,
                        color='red', marker='o', linewidth=2, markersize=4)
        axes[1, 0].set_title(
            'Principal Component Analysis: Explained Variance')
        axes[1, 0].set_xlabel('Principal Component')
        axes[1, 0].set_ylabel('Explained Variance Ratio')
        axes[1, 0].grid(True, alpha=0.3)

        pca_available = True

    except ImportError:
        axes[1, 0].text(0.5, 0.5, 'sklearn not available\nfor PCA analysis',
                        ha='center', va='center', transform=axes[1, 0].transAxes)
        axes[1, 0].set_title('Principal Component Analysis: Not Available')
        pca_available = False

    # 4. Correlation distribution
    # Get upper triangle correlation coefficients
    upper_triangle = correlation_matrix.where(
        np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))
    correlations = upper_triangle.stack().values

    axes[1, 1].hist(correlations, bins=20, alpha=0.7,
                    color='lightgreen', edgecolor='black')
    axes[1, 1].axvline(x=np.mean(correlations), color='red', linestyle='--',
                       label=f'Mean: {np.mean(correlations):.3f}')
    axes[1, 1].set_title('Inter-Tenor Correlation Distribution')
    axes[1, 1].set_xlabel('Correlation Coefficient')
    axes[1, 1].set_ylabel('Frequency')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    # Statistical summary
    print("\nCorrelation Analysis Results:")
    print(f"Average correlation: {np.mean(correlations):.3f}")
    print(f"Maximum correlation: {np.max(correlations):.3f}")
    print(f"Minimum correlation: {np.min(correlations):.3f}")

    if pca_available:
        print(f"1st PC explained variance: {explained_variance[0]:.3f}")
        if len(explained_variance) > 1:
            print(f"2nd PC explained variance: {explained_variance[1]:.3f}")

else:
    print("Correlation analysis requires at least 2 tenors")

## 4. Risk Parity Analysis

### Analysis Objective - 分析目的
- Construct portfolios that equalize risk contribution from each tenor
- Optimize risk-adjusted performance

### Methodology - 分析手法
- Calculate volatility for each tenor
- Calculate risk parity weights
- Backtest optimized portfolios
- Compare Sharpe ratios and risk metrics

### Interpretation - グラフの見方・解釈
- Equal weight vs Risk parity: Effect of risk adjustment
- Cumulative returns: Long-term performance comparison
- Drawdown: Maximum loss periods and magnitude

In [ ]:
# Risk parity analysis
if len(available_tenors) >= 2:
    # Calculate returns and volatility for each tenor
    returns_clean = returns_matrix.dropna()

    # Annualized volatility for each tenor
    volatilities = returns_clean.std() * np.sqrt(252)

    # Risk parity weights (inverse volatility)
    inverse_vol = 1 / volatilities
    risk_parity_weights = inverse_vol / inverse_vol.sum()

    # Equal weights
    equal_weights = np.ones(len(available_tenors)) / len(available_tenors)

    # Portfolio returns calculation
    equal_weighted_returns = (returns_clean * equal_weights).sum(axis=1)
    risk_parity_returns = (returns_clean * risk_parity_weights).sum(axis=1)

    # Cumulative returns calculation
    equal_weighted_cumulative = (1 + equal_weighted_returns).cumprod()
    risk_parity_cumulative = (1 + risk_parity_returns).cumprod()

    # Drawdown calculation
    def calculate_drawdown(returns):
        cumulative = (1 + returns).cumprod()
        running_max = cumulative.expanding().max()
        drawdown = (cumulative - running_max) / running_max
        return drawdown

    equal_weighted_dd = calculate_drawdown(equal_weighted_returns)
    risk_parity_dd = calculate_drawdown(risk_parity_returns)

    # Visualization
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))

    # 1. Weight comparison
    x = np.arange(len(available_tenors))
    width = 0.35

    bars1 = axes[0, 0].bar(x - width/2, equal_weights, width, label='Equal Weight',
                           alpha=0.7, color='lightblue', edgecolor='black')
    bars2 = axes[0, 0].bar(x + width/2, risk_parity_weights, width, label='Risk Parity',
                           alpha=0.7, color='lightcoral', edgecolor='black')

    axes[0, 0].set_title('Portfolio Weight Comparison')
    axes[0, 0].set_xlabel('Tenor')
    axes[0, 0].set_ylabel('Weight')
    axes[0, 0].set_xticks(x)
    axes[0, 0].set_xticklabels(
        [f'M{int(t)}' for t in available_tenors], rotation=45)
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)

    # 2. Cumulative returns comparison
    axes[0, 1].plot(equal_weighted_cumulative.index, equal_weighted_cumulative.values,
                    label='Equal Weight', linewidth=2, color='blue')
    axes[0, 1].plot(risk_parity_cumulative.index, risk_parity_cumulative.values,
                    label='Risk Parity', linewidth=2, color='red')
    axes[0, 1].set_title('Cumulative Returns Comparison')
    axes[0, 1].set_ylabel('Cumulative Return')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)

    # 3. Drawdown comparison
    axes[1, 0].fill_between(equal_weighted_dd.index, equal_weighted_dd.values, 0,
                            alpha=0.3, color='blue', label='Equal Weight')
    axes[1, 0].fill_between(risk_parity_dd.index, risk_parity_dd.values, 0,
                            alpha=0.3, color='red', label='Risk Parity')
    axes[1, 0].set_title('Drawdown Comparison')
    axes[1, 0].set_ylabel('Drawdown')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)

    # 4. Risk-return scatter plot
    individual_returns = returns_clean.mean() * 252
    individual_vol = volatilities

    axes[1, 1].scatter(individual_vol, individual_returns, s=100, alpha=0.7,
                       c='lightgreen', edgecolor='black', label='Individual Tenors')

    # Portfolio risk-return
    equal_vol = equal_weighted_returns.std() * np.sqrt(252)
    equal_ret = equal_weighted_returns.mean() * 252
    rp_vol = risk_parity_returns.std() * np.sqrt(252)
    rp_ret = risk_parity_returns.mean() * 252

    axes[1, 1].scatter(equal_vol, equal_ret, s=200, color='blue',
                       edgecolor='black', label='Equal Weight', marker='s')
    axes[1, 1].scatter(rp_vol, rp_ret, s=200, color='red',
                       edgecolor='black', label='Risk Parity', marker='^')

    axes[1, 1].set_xlabel('Annualized Volatility')
    axes[1, 1].set_ylabel('Annualized Return')
    axes[1, 1].set_title('Risk-Return Comparison')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    # Performance statistics
    def calculate_performance_metrics(returns):
        annual_return = returns.mean() * 252
        annual_vol = returns.std() * np.sqrt(252)
        sharpe_ratio = annual_return / annual_vol if annual_vol != 0 else 0
        max_drawdown = calculate_drawdown(returns).min()
        return {
            'Annual Return': annual_return,
            'Annual Volatility': annual_vol,
            'Sharpe Ratio': sharpe_ratio,
            'Max Drawdown': max_drawdown
        }

    equal_metrics = calculate_performance_metrics(equal_weighted_returns)
    rp_metrics = calculate_performance_metrics(risk_parity_returns)

    print("\nPortfolio Performance Comparison:")
    print("\nEqual Weight:")
    for key, value in equal_metrics.items():
        print(f"  {key}: {value:.4f}")

    print("\nRisk Parity:")
    for key, value in rp_metrics.items():
        print(f"  {key}: {value:.4f}")

    print("\nIndividual Tenor Volatilities:")
    for i, tenor in enumerate(available_tenors):
        if tenor in volatilities.index:
            print(f"  M{tenor}: {volatilities[tenor]:.4f}")

else:
    print("Risk parity analysis requires at least 2 tenors")

## 5. Anomaly Detection and Regime Analysis

### Analysis Objective - 分析目的
- Detect abnormal market movements and structural changes (regime changes)
- Improve risk management

### Methodology - 分析手法
- Statistical anomaly detection (Z-score, IQR method)
- Volatility regime detection
- Change point detection algorithms

### Interpretation - グラフの見方・解釈
- Outliers: Movements that greatly exceed normal price variations
- Regime changes: Persistent changes in volatility or correlation structure
- Change points: Market structure turning points

In [ ]:
# Anomaly detection and regime analysis
if len(available_tenors) > 0:
    main_tenor = available_tenors[0]
    price_series = price_pivot[main_tenor].dropna()
    returns = price_series.pct_change().dropna()

    # 1. Statistical anomaly detection
    # Z-score method
    z_scores = np.abs((returns - returns.mean()) / returns.std())
    z_threshold = 3.0
    z_outliers = z_scores > z_threshold

    # IQR method
    Q1 = returns.quantile(0.25)
    Q3 = returns.quantile(0.75)
    IQR = Q3 - Q1
    iqr_outliers = (returns < (Q1 - 1.5 * IQR)) | (returns > (Q3 + 1.5 * IQR))

    # 2. Volatility regime detection
    # 30-day rolling volatility
    rolling_vol = returns.rolling(window=30).std() * np.sqrt(252)
    vol_median = rolling_vol.median()

    # High and low volatility periods
    high_vol_regime = rolling_vol > vol_median * 1.5
    low_vol_regime = rolling_vol < vol_median * 0.5

    # Visualization
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))

    # 1. Outlier detection
    axes[0, 0].plot(returns.index, returns.values,
                    linewidth=0.5, color='blue', alpha=0.7)
    axes[0, 0].scatter(returns[z_outliers].index, returns[z_outliers].values,
                       color='red', s=50, alpha=0.8, label=f'Z-score > {z_threshold}')
    axes[0, 0].scatter(returns[iqr_outliers].index, returns[iqr_outliers].values,
                       color='orange', s=30, alpha=0.8, marker='x', label='IQR Outliers')
    axes[0, 0].set_title('Outlier Detection')
    axes[0, 0].set_ylabel('Daily Return')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)

    # 2. Volatility regime
    axes[0, 1].plot(rolling_vol.index, rolling_vol.values,
                    linewidth=1, color='blue')
    axes[0, 1].axhline(y=vol_median, color='green',
                       linestyle='--', alpha=0.7, label='Median')
    axes[0, 1].axhline(y=vol_median * 1.5, color='red',
                       linestyle='--', alpha=0.7, label='High Vol Threshold')
    axes[0, 1].axhline(y=vol_median * 0.5, color='orange',
                       linestyle='--', alpha=0.7, label='Low Vol Threshold')

    # Regime background coloring
    axes[0, 1].fill_between(rolling_vol.index, 0, rolling_vol.max(),
                            where=high_vol_regime, alpha=0.2, color='red', label='High Vol Regime')
    axes[0, 1].fill_between(rolling_vol.index, 0, rolling_vol.max(),
                            where=low_vol_regime, alpha=0.2, color='green', label='Low Vol Regime')

    axes[0, 1].set_title('Volatility Regime')
    axes[0, 1].set_ylabel('Annualized Volatility')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)

    # 3. Price level with outliers
    axes[1, 0].plot(price_series.index, price_series.values,
                    linewidth=1, color='blue')

    # Mark prices on outlier dates
    outlier_dates = returns[z_outliers | iqr_outliers].index
    if len(outlier_dates) > 0:
        outlier_prices = price_series.loc[outlier_dates]
        axes[1, 0].scatter(outlier_prices.index, outlier_prices.values,
                           color='red', s=50, alpha=0.8, label='Outlier Dates')

    axes[1, 0].set_title('Price Evolution with Outliers')
    axes[1, 0].set_ylabel('Price (USD)')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)

    # 4. Return distribution
    normal_returns = returns[~(z_outliers | iqr_outliers)]
    outlier_returns = returns[z_outliers | iqr_outliers]

    axes[1, 1].hist(normal_returns, bins=50, alpha=0.7, color='blue',
                    label=f'Normal (n={len(normal_returns)})', density=True)
    if len(outlier_returns) > 0:
        axes[1, 1].hist(outlier_returns, bins=20, alpha=0.7, color='red',
                        label=f'Outliers (n={len(outlier_returns)})', density=True)

    axes[1, 1].set_title('Return Distribution')
    axes[1, 1].set_xlabel('Daily Return')
    axes[1, 1].set_ylabel('Density')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    # Statistical summary
    print("\nOutlier Detection and Regime Analysis Results:")
    print(
        f"Z-score outliers: {z_outliers.sum()} ({z_outliers.sum()/len(returns)*100:.2f}%)")
    print(
        f"IQR outliers: {iqr_outliers.sum()} ({iqr_outliers.sum()/len(returns)*100:.2f}%)")
    print(
        f"High volatility periods: {high_vol_regime.sum()} days ({high_vol_regime.sum()/len(high_vol_regime)*100:.1f}%)")
    print(
        f"Low volatility periods: {low_vol_regime.sum()} days ({low_vol_regime.sum()/len(low_vol_regime)*100:.1f}%)")
    print(f"Volatility median: {vol_median:.4f}")
    print(f"Maximum volatility: {rolling_vol.max():.4f}")
    print(f"Minimum volatility: {rolling_vol.min():.4f}")

    # Largest outlier date
    if z_outliers.sum() > 0:
        max_outlier_date = returns[z_outliers].abs().idxmax()
        max_outlier_value = returns[max_outlier_date]
        print(
            f"\nLargest outlier: {max_outlier_date.strftime('%Y-%m-%d')} ({max_outlier_value:.4f})")

else:
    print("No data available for outlier detection analysis")

## Analysis Summary - 分析結果の総括

### Key Findings - 主要な発見
1. **Spread Trading**: Inter-month price differences show mean-reverting properties, indicating statistical arbitrage opportunities
2. **Seasonality**: Specific months or days showing bias suggest effective seasonal trading strategies
3. **Correlation Structure**: Inter-tenor correlations change over time, limiting diversification benefits
4. **Risk Management**: Risk parity strategies provide more stable performance than equal weighting
5. **Anomaly Detection**: Early detection of abnormal market movements enables improved risk management

### Practical Applications - 実践的な応用
- **Portfolio Construction**: Adoption of risk parity weights
- **Risk Management**: Early warning systems through anomaly detection
- **Trading Strategies**: Strategies combining seasonality and spreads
- **Dynamic Adjustment**: Portfolio rebalancing according to market regime changes

### Important Notes - 注意点
- Analysis based on historical data; does not guarantee future market performance
- Need to consider trading costs and liquidity constraints
- Past patterns may not continue due to market structural changes

In [ ]:
# Close database connection
conn.close()
print("Analysis complete. Database connection closed.")